In [ ]:
!pip install pinecone-client==3.2.2 sentence-transformers==2.7.0 datasets==2.19.0 transformers==4.40.1 openai==1.25.2 llama-index==0.10.34 llama-index-vector-stores-pinecone==0.1.6  -qqq
# faiss-cpu==1.7.2
!pip install -q condacolab
import condacolab
condacolab.install()

!conda install -c pytorch faiss-cpu -y

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 MB 51.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 75.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 763.0/763.0 kB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 792.7/792.7 kB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 766.7/766.7 MB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.1/150.1 MB 65.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 63.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 141.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.2/253.2 MB 71.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 94.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.

# **실습 데이터 다운로드**

In [ ]:
!wget ftp://ftp.irisa.fr/local/texmex/corpus/sift.tar.gz
!tar -xf sift.tar.gz
!mkdir data/sift1M -p
!mv sift/* data/sift1M

--2025-03-18 09:36:32--  ftp://ftp.irisa.fr/local/texmex/corpus/sift.tar.gz
           => ‘sift.tar.gz.2’
Resolving ftp.irisa.fr (ftp.irisa.fr)... 131.254.254.45, 2001:660:7303:254::45
Connecting to ftp.irisa.fr (ftp.irisa.fr)|131.254.254.45|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /local/texmex/corpus ... done.
==> SIZE sift.tar.gz ... 168280445
==> PASV ... done.    ==> RETR sift.tar.gz ... done.
Length: 168280445 (160M) (unauthoritative)

sift.tar.gz.2       100%[===================>] 160.48M  2.79MB/s    in 37s     

2025-03-18 09:37:11 (4.33 MB/s) - ‘sift.tar.gz.2’ saved [168280445]



# **실습 데이터 불러오기**

In [ ]:
import psutil

def get_memory_usage_mb():
  # 현재 실행 중인 프로세스 정보 가져오기
  process = psutil.Process()
  # 프로세스의 메모리 사용량 정보 가져오기
  memory_info = process.memory_info()

  # memory_info.rss -> Resident Set Size(RSS) -> 프로세스가 실제로 물리적 메모리에 로드된 크기
  # 바이트를 MB로 변환
  return memory_info.rss / (1024 * 1024)

In [ ]:
import time
import faiss
from faiss.contrib.datasets import DatasetSIFT1M

# DatasetSIFT1M -> FAISS에서 제공하는 SIFT1M 데이터셋을 로드하는 클래스
# SIFT1M -> 100만 개(1M) SIFT(Scale-Invariant Feature Transform) 벡터로 구성된 데이터셋
# 이미지 검색, 유사도 검색 등의 벤치마킹에 많이 사용됨
ds = DatasetSIFT1M()

# 검색에 사용할 데이터
xq = ds.get_queries()
# 저장된 벡터 데이터
xb = ds.get_database()
# 질문에 대한 실제 정답 데이터
gt = ds.get_groundtruth()

# **데이터가 늘어날 때 색인/검색 시간, 메모리 사용량 변화**

In [ ]:
k = 1
# 벡터 차원 추출 (128)
d = xq.shape[1]
nq = 1000
xq = xq[:nq]

# i를 2씩 증가
for i in range(1, 10, 2):
  start_memory = get_memory_usage_mb()
  start_indexing = time.time()

  index = faiss.IndexFlatL2(d)
  index.add(xb[:(i + 1) * 100000])

  end_indexing = time.time()
  end_memory = get_memory_usage_mb()

  t0 = time.time()
  D, I = index.search(xq, k)
  t1 = time.time()

  print(f"데이터 {(i + 1) * 100000}개:")
  print(f"색인: {(end_indexing - start_indexing) * 1000 :.3f} ms ({end_memory - start_memory:.3f} MB) 검색: {(t1 - t0) * 1000 / nq :.3f} ms")

ValueError: input not a numpy array

# **파라미터 m의 변경에 따른 성능 확인**

In [ ]:
import numpy as np

k = 1
d = xq.shape[1]
nq = 1000
xq = xq[:nq]

# 강제 변환 (안전하게)
xb = np.ascontiguousarray(xb, dtype=np.float32)
print(f"xb type: {type(xb)}")
print(f"xb dtype: {xb.dtype}")
print(f"xb shape: {xb.shape}")
print(f"Is xb C-contiguous? {xb.flags['C_CONTIGUOUS']}")

# xq = np.ascontiguousarray(xq, dtype=np.float32)
# print(f"xq type: {type(xq)}")
# print(f"xq dtype: {xq.dtype}")
# print(f"xq shape: {xq.shape}")
# print(f"Is xq C-contiguous? {xq.flags['C_CONTIGUOUS']}")

for m in [8, 16, 32, 64]:
  index = faiss.IndexHNSWFlat(d, m)

  time.sleep(3)

  start_memory = get_memory_usage_mb()
  start_index = time.time()

  try:
    index.add(xb)
    print("FAISS IndexHNSWFlat 생성 및 xb 추가 성공!")
  except ValueError as e:
    print(f"IndexHNSWFlat 인덱스 추가 중 에러 발생: {e}")

  end_memory = get_memory_usage_mb()
  end_index = time.time()

  print(f"M: {m} - 색인 시간: {end_index - start_index} s, 메모리 사용량: {end_memory - start_memory} MB")

  t0 = time.time()
  D, I = index.search(xq, k)
  t1 = time.time()

  # gt[:nq, :1] -> gt[:1000, :1] -> 즉 추린 질의 1000개 만큼의 정답 데이터 1000개를 가져와 실제 정답 인덱스인 1을 추출
  # 정답 데이터를 모두 더해서 전체 쿼리 개수로 나눠 계산한다
  recall_at_1 = np.equal(I, gt[:nq, :1]).sum() / float(nq)
  print(f"{(t1 - t0) * 1000.0 / nq:.3f} ms per query, R@1 {recall_at_1:.3f}")

xb type: <class 'numpy.ndarray'>
xb dtype: float32
xb shape: (1000000, 128)
Is xb C-contiguous? True
FAISS IndexHNSWFlat 생성 및 xb 추가 성공!
M: 8 - 색인 시간: 111.28687405586243 s, 메모리 사용량: 752.109375 MB
0.040 ms per query, R@1 0.697
FAISS IndexHNSWFlat 생성 및 xb 추가 성공!
M: 16 - 색인 시간: 120.2768120765686 s, 메모리 사용량: 632.86328125 MB
0.051 ms per query, R@1 0.785
FAISS IndexHNSWFlat 생성 및 xb 추가 성공!
M: 32 - 색인 시간: 233.6597261428833 s, 메모리 사용량: 755.3828125 MB
0.097 ms per query, R@1 0.904
FAISS IndexHNSWFlat 생성 및 xb 추가 성공!
M: 64 - 색인 시간: 297.6750257015228 s, 메모리 사용량: 999.16015625 MB
0.220 ms per query, R@1 0.934


# **ef_construction을 변화시킬 때 성능 확인**

In [ ]:
k = 1
d = xq.shape[1]
nq = 1000
xq = xq[:nq]

for ef_construction in [40, 80, 160, 320]:
  index = faiss.IndexHNSWFlat(d, 32)
  index.hnsw.efConstruction = ef_construction

  time.sleep(3)

  start_memory = get_memory_usage_mb()
  start_index = time.time()

  index.add(xb)

  end_memory = get_memory_usage_mb()
  end_index = time.time()

  print(f"efConstruction: {ef_construction} - 색인 시간: {end_index - start_index} s, 메모리 사용량: {end_memory - start_memory} MB")



# **ef_search 변경에 따른 성능 확인**

In [ ]:
for ef_search in [16, 32, 64, 128]:
  index.hnsw.efSearch = ef_search

  t0 = time.time()

  D, I = index.search(xq, k)

  t1 = time.time()

  recall_at_1 = np.equal(I, gt[:nq, :1]).sum() / float(nq)

  print(f"{(t1 - t0) * 1000.0 / nq:.3f} ms per query, R@1 {recall_at_1:.3f}")

# **파인콘 계정 연결 및 인덱스 생성**

In [ ]:
# ServerlessSpec -> 서버리스 인덱스 설정
from pinecone import Pinecone, ServerlessSpec

pinecone_api_key = "자신의 API 키를 입력"
pc = Pinecone(api_key=pinecone_api_key)

# ServerlessSpec("aws", "us-east-1") -> Pinecone의 서버리스 모드에서 인덱스를 AWS의 us-east-1 리전에 생성
# 꼭 서버리스로 사용하지 않아도 된다
pc.create_index("llm-book", spec=ServerlessSpec("aws", "us-east-1"), dimension=768)
index = pc.index('llm-book')

# **임베딩 생성**

In [ ]:
from datasets import load_dataset
from sentence_transformers import SentenceTransformer

# 임베딩 모델 불러오기
sentence_model = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')

# 데이터셋 불러오기
klue_dp_train = load_dataset('klue', 'dp', split='train')

embeddings = sentence_model.encode(klue_dp_train['sentence'])

# **파인콘 입력을 위한 데이터 형태 변경**

In [ ]:
# 파이썬 기본 데이터 타입으로 변경
embeddings = embedding.tolist()

# {"id": 문서 ID(str), "values": 벡터 임베딩(List[float]), "metadata": 메타 데이터(dict) ) 형태로 데이터 준비
insert_data = []

for idx, (embedding, text) in enumerate(zip(embeddings, klue_dp_train['sentence'])):
  insert_data.append({"id": str(idx), "values": embedding, "metadata": {'text': text}})

# **임베딩 데이터를 인덱스에 저장**

In [ ]:
upsert_response = index.upsert(vectors = insert_data, namespace='llm-book-sub')

# **인덱스 검색하기**

In [ ]:
query_response = index.query(
    namespace='llm-book-sub', # 검색할 네임스페이스
    top_k=10, # 몇 개의 결과를 반환할지
    include_values=True, # 벡터 임베딩 반환 여부
    include_meatadata=True, # 메타 데이터 반환 여부
    vector=embeddings[0] # 검색할 벡터 임베딩
)

query_response

# **파인콘에서 문서 수정 및 삭제**

In [ ]:
new_text = '변경할 새로운 텍스트'
new_embedding = sentence_model.encode(new_text).tolist()

# 업데이트
update_response = index.update(
    id='기존_문서_id',
    values=new_embedding,
    set_metadata={'text': new_text},
    namespace='llm-book-sub'
)

# 삭제
delete_response = index.delete(ids=['기존_문서_id'], namespace='llm-book-sub')